## Example: Seinfeld Food Database


In [ ]:
import ibis
import os
from IPython.display import display, HTML
from markdown import markdown as md
DATADIR = "/home/shared"
from cdsutils.mutils import *
from cdsutils.sqlite import *
from dminteract.creator.utils import *

In [ ]:
!cp /home/shared/seinfeld_utf8.sqlite3 .

In [ ]:
conn = ibis.sqlite.connect("./seinfeld_utf8.sqlite3")
conn.list_tables()

In [ ]:
tft = conn.table("food_types")

In [ ]:
print(ibis.sqlite.compile(tft))

## What do these tables look like?
### Look at the Schema

In [ ]:
for t in conn.list_tables():
    tbl = conn.table(t)
    nrows = tbl.count().execute()
    display(HTML(md("### Table: %s\n\nnumber of rows=%3d\n####Schema"%(t, nrows))))
    display(view_dict(tbl.schema(), vertical=False))

### Look at the actual data

# Putting Data Back Together

Since this is a relational database, data has been split across tables so that the data is in __First Normal Form__. To make the data useful we have to put the data back together with __joins__.

## Example: _What Food is in Each Episode?_

### What tables do we need to put together?

- `foods_episodes`: lists `food_id` and `episode_id`

Let's try joining `food` with `food_id` so we can see the names of the food

#### Problem!

Before we start, we need to recognize that we've got a problem. The columns in the tables have overlapping names:

- Every table has a column named `id`
- `foods`, `episodes` and `food_types` all have columns named `name`

#### Create table objects for our two tables

In [ ]:
tf = conn.table("foods")
tfe = conn.table("foods_episodes")

### Join the tables

To join the tables we use the `join` method for which we need to specify which columns in each table refer to each other. For example `food_id` in `foods_episodes` refers to `id` in `foods`.

Both tables have a column named `id` so we will either have to discard one or both of them or rename one or both of them. To start with let's just discard the `id` columns. We will do this by specifically listing what columns to keep. 

In [ ]:
tnew = tfe.join(tf, tfe.food_id==tf.id).projection([tf.name, tfe.episode_id])
itview(tnew)

#### Only Half way there

To make this useful, we still need to replace episode_id with something more useful. We need to join with `episodes`. `episode_id` refers to `id` in `episodes`.

Now we have to face the overlapping names, namely `name`. We can use the `name()` method to rename columns.

In [ ]:
tf = conn.table("foods")
tfe = conn.table("foods_episodes")
te = conn.table("episodes")


tnew = tfe.join(tf, tfe.food_id==tf.id).join(te, tfe.episode_id==te.id).projection([tf.name, te.name.name("episode_name"), tfe.episode_id, te.season])
itview(tnew)

In [ ]:
tnew.episode_id.distinct().count().execute(), tnew.episode_name.distinct().count().execute()

## A Note on Line breaks in Python

The line above where we define `tnew` was very long, hence unreadable, and poor style. So we want to break the line up. However, Python is sensitive to indentation, so you to have careful with how lines are broken up. Here are two different ways we can break the lines up.

In [ ]:
tnew = tfe.join(
    tf, tfe.food_id==tf.id).join(
        te, tfe.episode_id==te.id).projection(
            [tf.name, te.name.name("episode_name"), 
             tfe.episode_id, te.season])


In [ ]:
tnew = tfe\
.join(tf, tfe.food_id==tf.id)\
.join(te, tfe.episode_id==te.id)\
.projection([tf.name, te.name.name("episode_name"), 
             tfe.episode_id, te.season])

__I kept both `tfe.episode_id` and `te.id` to demonstrate that they really are the same__

#### What are we still missing?

- The type of food
- `join` with `food_types`
    - `foods.type_id` refers to `food_types.id`
    - rename `food_types.name` to `food_category`

In [ ]:
tf = conn.table("foods")
tfe = conn.table("foods_episodes")
te = conn.table("episodes")
tft = conn.table("food_types")

tnew = tfe.join(tf, tfe.food_id==tf.id)\
.join(te, tfe.episode_id==te.id)\
.join(tft, tft.id==tf.type_id)\
.projection(
    [tf.name, tft.name.name("food_category"),
     te.name.name("episode_name"), 
     tfe.episode_id, te.season, ])
itview(tnew)

#### `sort`

The order of our results seem less than ideal. We can sort our results using the `sort` method. `name` also seems too generic, so let's rename it to `food_name`.

In [ ]:
tfe = conn.table("foods_episodes")
tf = conn.table("foods")
te = conn.table("episodes")
tft = conn.table("food_types")
tnew = te.join(tfe, tfe.episode_id== te.id)\
.join(tf, tfe.food_id==tf.id)\
.join(tft, tf.type_id==tft.id)\
.projection(
    [tf.name.name("food_name"),
     tft.name.name("food_category"),
     te.name.name("episode_name"), 
     tfe.episode_id, 
     te.season]).sort_by("episode_id")

In [ ]:
itview(tnew)

# Filtering Data (`filter`/ SQL WHERE)

Now that we know how to put things together, we need to look at how to keep the parts we are interested in and discarding the rest. In Ibis this is [`filtering`](https://docs.ibis-project.org/notebooks/tutorial/2-Basics-Aggregate-Filter-Limit.html#Filtering) (corresponding to [`WHERE`](https://bit.ly/2Nv927e) in SQL).

We execute a filter by passing a list of expressions that evaluate to True or False.

__*Very Important Note*__: In Python if I want to ask if two things are equal, the symbol is __`==`__. A single __`=`__ is an assignment statement, not a question about equality.

#### Example

Select all the rows that include `Junkfood`

In [ ]:
itview(tnew.filter([tnew.food_category == "Junkfood"]))

In [ ]:
itview(tnew.filter([tnew.food_category == "Junkfood",
                    tnew.season < 6]))

### More Filters

This dataset is relatively simple in that there are only two types of data:

- Integers (e.g. `episode_id`, `season`)
- Strings (e.g. `food_name`, `food_category`)

Filtering with integers is straightroward (e.g. ==, >, <)

Filtering with strings is more complex.

#### Filtering with Strings: Partial Matching

- __Strings are case sensitive:__ "Mint" is not equal to "mint"
    - __Tip__: Consider convert everything to uppercase or lowercase
- __Pattern Matching:__ A variety of options here
    - `contains()`
        - Does the string contain the 
    - `like`
        - This uses wildcard matches: 
            - **_** matches any one character
            - __%__ matches any one OR MORE character(s)
    - `rlike`
        - This uses regular expressions. Regular expressions are VERY powerful and VERY trick and so VERY frustrating
        - For those who are interested, I'll provide regular expression  materials later.

#### Example: Finding Minty Foods

In [ ]:
tnew.filter([tnew.food_name.contains("mint")]).execute()

In [ ]:
tnew.filter([tnew.food_name.lower().contains("mint")]).execute()

#### Example: Like

Find all foods that start with "P" in season 5.

In [ ]:
tnew.filter([tnew.food_name.like("P%"),tnew.season==5]).execute()

### Negating Filters

- You can negate a filter by prepending it with a negative sign (__-__)

In [ ]:
tnew.filter([-tnew.food_name.like("P%"),tnew.season==5]).execute()

### Introducing `count`

Often times we just want to see how many rows there are that match a condition.

Ibis provides a `count` method that will count the number of resuslting rows.

So what if we want to count the number of foods with a capital `P`.

In [ ]:
tnew.filter([tnew.food_name.like("%P%"),tnew.season==5]).count().execute()

### What if we want to see how many `p`s there are, upper or lower?

Remember we have the `upper` and `lower` methods to convert strings to uppercase or lowercase.

In [ ]:
tnew.filter([tnew.food_name.upper().like("%P%"),tnew.season==5]).count().execute()

In [ ]:
for w in create_question_bank("day1_bec.yaml", tag="ibis, food, case"):
    display(w)

### `AND` and `OR`

Providing multiple filters acts like a logical `AND`:

"Show me all the rows where `food_name` has a `P` __AND__ the eposide is in season 6."

So if I provide two mutually exclusive filters, I will get no results.

"Show me all the rows where an episode is in season 5 __AND__ the episode is in season 6."

In [ ]:
tnew.filter([tnew.season==5,tnew.season==6]).execute()

### `OR` in Ibis

- An OR question is something like:
 - "Show me all the rows where there is a food name with a `P` and the episode is in season 5 __OR__ in season 6.

- I can create an `OR` with the symbol __`|`__.
    - There is a similar `AND` operator: `&`

__NOTE__: I had to wrap each question for the OR in parentheses so that the `|` operator didn't try to apply to the number 5 and `tnew.season`.



In [ ]:
tnew.filter([tnew.food_name.like("%P%"),(tnew.season==5) | (tnew.season==6)]).execute()

If I pass a list of conditions to `like` this acts like an __OR__

In [ ]:
tnew.filter([tnew.food_name.like(["%Pepsi%", "%Cola%"])]).execute()

In [ ]:
for w in create_question_bank("day1_bec.yaml", tag="ibis,food,like"):
    display(w)


## Getting Distinct/Unique Values

Remember that in relational databases, we are aiming for at least 1st normal form: one distince value in each cell. So in our results we often have multiple rows with the same episode or the same food. What if we want to count the number of episodes that mention pie? Then we can use the `distinct` method.

In [ ]:
tnew.filter(tnew.food_name.like("%pie%")).episode_id.distinct().execute()

In [ ]:
tnew.filter([tnew.food_name.like("%P%"),(tnew.season==5) | (tnew.season==6)]).episode_id.distinct().count().execute()

## Missing Values

When we execute an Ibis query, it returns a Pandas dataframe. In Pandas `NaN` indicates a missing value. In this data set,the _Pilot_, the very first episode of the TV show is not in a season so the `season` value is `NaN`.

In [ ]:
tnew.execute(limit=5)

### `notnull`

Ibis provides a `notnull` method that allows us to filter by out missing values.

In [ ]:
tnew.filter([tnew.food_category=="Drinks", tnew.season.notnull()]).sort_by("food_name").execute()

In [ ]:
tnew.filter([tnew.season.notnull()]).sort_by("food_name").execute(limit=10)

## Practice

In [ ]:
for w in create_question_bank("day1_bec.yaml", tag="ibis,practice"):
    display(w)